# Get log probs

Certain chat models can be configured to return token-level log probabilities. This guide walks through how to get logprobs for a number of models.

In [1]:
%pip install -qU langchain-openai

Note: you may need to restart the kernel to use updated packages.


In [1]:
from langchain_openai import ChatOpenAI

# For the OpenAI API to return log probabilities we need to configure the `logprobs` param
llm = ChatOpenAI(model="gpt-3.5-turbo-0125").bind(logprobs=True)

msg = llm.invoke(("human", "how are you today"))

In [2]:
msg.additional_kwargs["logprobs"]["content"][:5]

[{'token': 'I', 'bytes': [73], 'logprob': -0.21393852, 'top_logprobs': []},
 {'token': "'m",
  'bytes': [39, 109],
  'logprob': -0.3048596,
  'top_logprobs': []},
 {'token': ' just',
  'bytes': [32, 106, 117, 115, 116],
  'logprob': -0.23057781,
  'top_logprobs': []},
 {'token': ' a',
  'bytes': [32, 97],
  'logprob': -0.0018863997,
  'top_logprobs': []},
 {'token': ' computer',
  'bytes': [32, 99, 111, 109, 112, 117, 116, 101, 114],
  'logprob': -0.049347106,
  'top_logprobs': []}]

In [3]:
ct = 0
full = None
for chunk in llm.stream(("human", "how are you today")):
    if ct < 5:
        full = chunk if full is None else full + chunk
        print(full.additional_kwargs["logprobs"]["content"])
    else:
        break
    ct += 1

[]
[{'token': 'I', 'bytes': [73], 'logprob': -0.21393852, 'top_logprobs': []}]
[{'token': 'I', 'bytes': [73], 'logprob': -0.21393852, 'top_logprobs': []}, {'token': "'m", 'bytes': [39, 109], 'logprob': -0.34287196, 'top_logprobs': []}]
[{'token': 'I', 'bytes': [73], 'logprob': -0.21393852, 'top_logprobs': []}, {'token': "'m", 'bytes': [39, 109], 'logprob': -0.34287196, 'top_logprobs': []}, {'token': ' an', 'bytes': [32, 97, 110], 'logprob': -2.468603, 'top_logprobs': []}]
[{'token': 'I', 'bytes': [73], 'logprob': -0.21393852, 'top_logprobs': []}, {'token': "'m", 'bytes': [39, 109], 'logprob': -0.34287196, 'top_logprobs': []}, {'token': ' an', 'bytes': [32, 97, 110], 'logprob': -2.468603, 'top_logprobs': []}, {'token': ' AI', 'bytes': [32, 65, 73], 'logprob': -0.016838558, 'top_logprobs': []}]


In [17]:
def multiply(a: int, b: int) -> int:
    """mulitply"""
    return a * b


full = None
chunks = []
for chunk in (
    ChatOpenAI(model="gpt-3.5-turbo-0125").bind_tools([multiply]).stream("what's 3 * 5")
):
    full = chunk if full is None else full + chunk
    print(chunk)
    # print(full)

content='' additional_kwargs={'tool_calls': [{'index': 0, 'id': 'call_huirg8sWQiT0AR1wqmgs5sqE', 'function': {'arguments': '', 'name': 'multiply'}, 'type': 'function'}]}
content='' additional_kwargs={'tool_calls': [{'index': 0, 'id': None, 'function': {'arguments': '{"', 'name': None}, 'type': None}]}
content='' additional_kwargs={'tool_calls': [{'index': 0, 'id': None, 'function': {'arguments': 'a', 'name': None}, 'type': None}]}
content='' additional_kwargs={'tool_calls': [{'index': 0, 'id': None, 'function': {'arguments': '":', 'name': None}, 'type': None}]}
content='' additional_kwargs={'tool_calls': [{'index': 0, 'id': None, 'function': {'arguments': '3', 'name': None}, 'type': None}]}
content='' additional_kwargs={'tool_calls': [{'index': 0, 'id': None, 'function': {'arguments': ',"', 'name': None}, 'type': None}]}
content='' additional_kwargs={'tool_calls': [{'index': 0, 'id': None, 'function': {'arguments': 'b', 'name': None}, 'type': None}]}
content='' additional_kwargs={'tool

In [6]:
full

AIMessageChunk(content='', additional_kwargs={'tool_calls': [{'index': 0, 'id': 'call_LQXQMkkgUVr9FNJpgCXfeHhI', 'function': {'arguments': '{"aa":":33,","bb":":55}}', 'name': 'multiply'}, 'type': 'function'}], 'finish_reason': 'tool_calls'})

In [7]:
from langchain_core.utils._merge import merge_dicts

merge_dicts(
    {
        "tool_calls": [
            {
                "index": 0,
                "id": "call_LQXQMkkgUVr9FNJpgCXfeHhI",
                "function": {"arguments": '{"', "name": "multiply"},
                "type": "function",
            }
        ]
    },
    {
        "tool_calls": [
            {
                "index": 0,
                "id": None,
                "function": {"arguments": "a", "name": None},
                "type": None,
            }
        ]
    },
)

{'tool_calls': [{'index': 0,
   'id': 'call_LQXQMkkgUVr9FNJpgCXfeHhI',
   'function': {'arguments': '{"a', 'name': 'multiply'},
   'type': 'function'}]}

In [8]:
merge_dicts(
    {
        "tool_calls": [
            {
                "index": 0,
                "id": "call_LQXQMkkgUVr9FNJpgCXfeHhI",
                "function": {"arguments": '{"a', "name": "multiply"},
                "type": "function",
            }
        ]
    },
    {
        "tool_calls": [
            {
                "index": 0,
                "id": None,
                "function": {"arguments": '":', "name": None},
                "type": None,
            }
        ]
    },
)

{'tool_calls': [{'index': 0,
   'id': 'call_LQXQMkkgUVr9FNJpgCXfeHhI',
   'function': {'arguments': '{"a":', 'name': 'multiply'},
   'type': 'function'}]}

In [12]:
from langchain_core.messages import AIMessageChunk

(
    AIMessageChunk(
        "",
        additional_kwargs={
            "tool_calls": [
                {
                    "index": 0,
                    "id": "call_LQXQMkkgUVr9FNJpgCXfeHhI",
                    "function": {"arguments": '{"a', "name": "multiply"},
                    "type": "function",
                }
            ]
        },
    )
    + AIMessageChunk(
        "",
        additional_kwargs={
            "tool_calls": [
                {
                    "index": 0,
                    "id": None,
                    "function": {"arguments": '":', "name": None},
                    "type": None,
                }
            ]
        },
    )
)

AIMessageChunk(content='', additional_kwargs={'tool_calls': [{'index': 0, 'id': 'call_LQXQMkkgUVr9FNJpgCXfeHhI', 'function': {'arguments': '{"a":', 'name': 'multiply'}, 'type': 'function'}]})

In [21]:
for chunk in (
    ChatOpenAI(model="gpt-3.5-turbo-0125").bind_tools([multiply]).stream("what's 3 * 5")
):
    print(chunk)

content=''
content='' additional_kwargs={'tool_calls': [{'index': 0, 'id': 'call_ditdM856HaUDKioBsVuMk7iq', 'function': {'arguments': '', 'name': 'multiply'}, 'type': 'function'}]}
content='' additional_kwargs={'tool_calls': [{'index': 0, 'id': None, 'function': {'arguments': '{"a"', 'name': None}, 'type': None}]}
content='' additional_kwargs={'tool_calls': [{'index': 0, 'id': None, 'function': {'arguments': ': 3, ', 'name': None}, 'type': None}]}
content='' additional_kwargs={'tool_calls': [{'index': 0, 'id': None, 'function': {'arguments': '"b": 5', 'name': None}, 'type': None}]}
content='' additional_kwargs={'tool_calls': [{'index': 0, 'id': None, 'function': {'arguments': '}', 'name': None}, 'type': None}]}
content='' additional_kwargs={'finish_reason': 'tool_calls'}


In [22]:
chunks = list(
    ChatOpenAI(model="gpt-3.5-turbo-0125").bind_tools([multiply]).stream("what's 3 * 5")
)
chunks

[AIMessageChunk(content='', additional_kwargs={'tool_calls': [{'index': 0, 'id': 'call_h9NhPi6irvmQAxOXLGw7CF7d', 'function': {'arguments': '{"a":3,"b":5}', 'name': 'multiply'}, 'type': 'function'}]}),
 AIMessageChunk(content='', additional_kwargs={'tool_calls': [{'index': 0, 'id': None, 'function': {'arguments': '{"', 'name': None}, 'type': None}]}),
 AIMessageChunk(content='', additional_kwargs={'tool_calls': [{'index': 0, 'id': None, 'function': {'arguments': 'a', 'name': None}, 'type': None}]}),
 AIMessageChunk(content='', additional_kwargs={'tool_calls': [{'index': 0, 'id': None, 'function': {'arguments': '":', 'name': None}, 'type': None}]}),
 AIMessageChunk(content='', additional_kwargs={'tool_calls': [{'index': 0, 'id': None, 'function': {'arguments': '3', 'name': None}, 'type': None}]}),
 AIMessageChunk(content='', additional_kwargs={'tool_calls': [{'index': 0, 'id': None, 'function': {'arguments': ',"', 'name': None}, 'type': None}]}),
 AIMessageChunk(content='', additional_k

In [24]:
chunks[1] + chunks[2] + chunks[3]

AIMessageChunk(content='', additional_kwargs={'tool_calls': [{'index': 0, 'id': None, 'function': {'arguments': '{"aa":', 'name': None}, 'type': None}]})